In [1]:
#Importing the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
# Setting the format type explicitly to avoid scientific notation 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
# Reading the csv file and printing the shape of the data set
loan_master = pd.read_csv("loan.csv", low_memory = False)
print("The initial data set comprises of",loan_master.shape[0],"records and", loan_master.shape[1],"columns")

The initial data set comprises of 39717 records and 111 columns


### Data Understanding

In [37]:
#Header section of the master dataframe
loan_master.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.000,36 months,10.65%,162.870,B,B2,NaN,10+ years,RENT,24000.000,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.650,0,Jan-85,1,nan,nan,3,0,13648,83.70%,9,f,0.000,0.000,5863.155,5833.840,5000.000,863.160,0.000,0.000,0.000,Jan-15,171.620,NaN,May-16,0.000,nan,1,INDIVIDUAL,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan
1,1077430,1314167,2500,2500,2500.000,60 months,15.27%,59.830,C,C4,Ryder,< 1 year,RENT,30000.000,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.000,0,Apr-99,5,nan,nan,3,0,1687,9.40%,4,f,0.000,0.000,1008.710,1008.710,456.460,435.170,0.000,117.080,1.110,Apr-13,119.660,NaN,Sep-13,0.000,nan,1,INDIVIDUAL,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan


#### Since the objective of this EDA is to identify the driver variables that lead to default loans, we can consider loan_status as the dependent variable and all the other variables as the independent variables which can have a varying influence on the loan_status

In [38]:
# Value counts for the loan_status variable
loan_master.loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [39]:
# Since we are interested in identifying patterns that related to either Fully Paid loand or Charged off loans, we 
# can filter out the rows where loan_status is Current and naming the resultant dataframe as df_sub
loan_master = loan_master.loc[loan_master['loan_status']!= 'Current']

In [40]:
# Printing the shape of the dataframe containing values where loan status is either 'Fully Paid' or 'Current'
print(loan_master.shape) 

(38577, 111)


### Data Cleaning

In [42]:
#Checking for the percentage of null values in loan_master
round(100*(loan_master.isnull().sum()/len(loan_master.index)), 2)

id                                 0.000
member_id                          0.000
loan_amnt                          0.000
funded_amnt                        0.000
funded_amnt_inv                    0.000
term                               0.000
int_rate                           0.000
installment                        0.000
grade                              0.000
sub_grade                          0.000
emp_title                          6.180
emp_length                         0.000
home_ownership                     0.000
annual_inc                         0.000
verification_status                0.000
issue_d                            0.000
loan_status                        0.000
pymnt_plan                         0.000
url                                0.000
desc                              32.470
purpose                            0.000
title                              0.030
zip_code                           0.000
addr_state                         0.000
dti             

In [43]:
mis_100 = (round(100*(loan_master.isnull().sum()/len(loan_master.index)), 2) == 100)
print("The number of variables having 100% of their values as missing are",len(loan_master.columns[mis_100].tolist()))

The number of variables having 100% of their values as missing are 55


In [44]:
#Dropping the 54 variables where all or 100% the values are missing and creating a new dataframe loan_clean
loan_clean = loan_master.dropna(axis = 1, how = 'all')

In [45]:
# Printing the shape of the dataset after the 54 variables containing all null values have been dropped
print("After dropping the 54 variables containing 100% of their values as null we have",loan_clean.shape[0],"records and", loan_clean.shape[1],"columns in the new dataframe loan_clean")

After dropping the 54 variables containing 100% of their values as null we have 38577 records and 56 columns in the new dataframe loan_clean


In [46]:
# Identifying the list of columns where the percentage of missing values is greater than 30%
mis_30 = (round(100*(loan_clean.isnull().sum()/len(loan_clean.index)), 2) >30)

In [47]:
print("The variables having greater than 30% of missing values are",loan_clean.columns[mis_30].tolist())

The variables having greater than 30% of missing values are ['desc', 'mths_since_last_delinq', 'mths_since_last_record']


In [48]:
loan_clean.drop(loan_clean.columns[mis_30].tolist(), axis = 1, inplace = True)

/Users/ashishveera/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
print("After dropping the 4 variables containing greater than 30% of their values as null we have",loan_clean.shape[0],"records and", loan_clean.shape[1],"columns")

After dropping the 4 variables containing greater than 30% of their values as null we have 38577 records and 53 columns


In [50]:
# Checking if any of the rows contain all null values
loan_clean.isnull().all(axis=1).sum()

0

In [51]:
print("Number of records having atleast 1 missing value:",loan_clean[loan_clean.isnull().sum(axis =1)>1].shape[0])
print("Number of records having atleast 2 missing values:",loan_clean[loan_clean.isnull().sum(axis =1)>2].shape[0])
print("Number of records having atleast 3 missing values:",loan_clean[loan_clean.isnull().sum(axis =1)>3].shape[0])
print("Number of records having atleast 4 missing values:",loan_clean[loan_clean.isnull().sum(axis =1)>4].shape[0])
print("Number of records having atleast 5 missing values:",loan_clean[loan_clean.isnull().sum(axis =1)>5].shape[0])

Number of records having atleast 1 missing value: 108
Number of records having atleast 2 missing values: 55
Number of records having atleast 3 missing values: 41
Number of records having atleast 4 missing values: 7
Number of records having atleast 5 missing values: 0


In [52]:
# Understanding the count of unique values in the dataframe
loan_clean.nunique()

id                            38577
member_id                     38577
loan_amnt                       870
funded_amnt                    1019
funded_amnt_inv                8050
term                              2
int_rate                        370
installment                   15022
grade                             7
sub_grade                        35
emp_title                     28028
emp_length                       12
home_ownership                    5
annual_inc                     5215
verification_status               3
issue_d                          55
loan_status                       2
pymnt_plan                        1
url                           38577
purpose                          14
title                         19298
zip_code                        822
addr_state                       50
dti                            2853
delinq_2yrs                      11
earliest_cr_line                524
inq_last_6mths                    9
open_acc                    

#### As shown above, the data frame consists of 11 variables where all the corresponding values are unique. Since variables having unique values will not have a significant difference in means across the two populations which is the records having their corresponding loan status as Fully Paid vs records having loan status as Charged off. It is therefore safe to drop these unique value columns since these values are more like constants would not influence the value of the dependent varable loan_status

In [53]:
# Dropping the variables where the count of unique values is equal to 1. 
unique_cols = loan_clean.nunique()==1
loan_clean.drop(loan_clean.columns[unique_cols].tolist(), axis = 1, inplace = True)

/Users/ashishveera/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
print("After dropping the unique value columns, the remaining number of columns are", loan_clean.shape[1], "and the number of records reamining are", loan_clean.shape[0])

After dropping the unique value columns, the remaining number of columns are 42 and the number of records reamining are 38577


In [55]:
loan_clean.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'url', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')

#### So far the loan dataframe has been cleaned basing on the issues with the null values. Now let's filter the data set basing on the business understanding of this use case

In [56]:
loan_clean.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,1077501,1296599,5000,5000,4975.000,36 months,10.65%,162.870,B,B2,NaN,10+ years,RENT,24000.000,Verified,Dec-11,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.650,0,Jan-85,1,3,0,13648,83.70%,9,5863.155,5833.840,5000.000,863.160,0.000,0.000,0.000,Jan-15,171.620,May-16,0.000
1,1077430,1314167,2500,2500,2500.000,60 months,15.27%,59.830,C,C4,Ryder,< 1 year,RENT,30000.000,Source Verified,Dec-11,Charged Off,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.000,0,Apr-99,5,3,0,1687,9.40%,4,1008.710,1008.710,456.460,435.170,0.000,117.080,1.110,Apr-13,119.660,Sep-13,0.000


In [57]:
col_drop = ['id','member_id','url','zip_code']
loan_clean.drop(col_drop, axis =1, inplace = True)

/Users/ashishveera/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [58]:
print("After dropping the columns which are not needed from a business perspective, the remaining number of columns are", loan_clean.shape[1], "and the number of records reamining are", loan_clean.shape[0])

After dropping the columns which are not needed froma business perspective, the remaining number of columns are 38 and the number of records reamining are 38577


In [59]:
loan_clean.dtypes

loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
emp_title                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
purpose                     object
title                       object
addr_state                  object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec                      int64
revol_bal                    int64
revol_util                  object
total_acc                    int64
total_pymnt                float64
total_pymnt_inv     

In [61]:
loan_clean.nunique()

loan_amnt                    870
funded_amnt                 1019
funded_amnt_inv             8050
term                           2
int_rate                     370
installment                15022
grade                          7
sub_grade                     35
emp_title                  28028
emp_length                    12
home_ownership                 5
annual_inc                  5215
verification_status            3
issue_d                       55
loan_status                    2
purpose                       14
title                      19298
addr_state                    50
dti                         2853
delinq_2yrs                   11
earliest_cr_line             524
inq_last_6mths                 9
open_acc                      40
pub_rec                        5
revol_bal                  21275
revol_util                  1088
total_acc                     82
total_pymnt                36714
total_pymnt_inv            36387
total_rec_prncp             6841
total_rec_